In [1]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm




In [2]:
data1 = np.load('data1.npy')
lab1 = np.load('lab1.npy')
data0 = np.load('data0.npy')
lab0 = np.load('lab0.npy')
data2 = np.load('data2.npy')
lab2 = np.load('lab2.npy')
# Load data
data = np.concatenate([data0, data1, data2], axis=0)
labels = np.concatenate([lab0, lab1, lab2], axis=0)

# Check data shapes
print(f'Data shape: {data.shape}')
print(f'Labels shape: {labels.shape}')

Data shape: (30000, 40, 168)
Labels shape: (30000,)


In [3]:
# Split into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Further split train data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)

print(f'Train data shape: {train_data.shape}')
print(f'Validation data shape: {val_data.shape}')
print(f'Test data shape: {test_data.shape}')


Train data shape: (21600, 40, 168)
Validation data shape: (2400, 40, 168)
Test data shape: (6000, 40, 168)


In [4]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            sample = self.transform(sample)

        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Define transform for grayscale images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Repeat the single channel 3 times
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Update the CustomDataset to use the transform
train_dataset = CustomDataset(train_data, train_labels, transform=transform)
val_dataset = CustomDataset(val_data, val_labels, transform=transform)
test_dataset = CustomDataset(test_data, test_labels, transform=transform)

# Update DataLoaders
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 37)  # 37 is the number of classes

model = model.to(device)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)  # Wrap model in DataParallel


/home2/sanchit27/miniconda3/envs/sanchit/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home2/sanchit27/miniconda3/envs/sanchit/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using 4 GPUs!


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50):
    best_model = None
    best_val_accuracy = 0.0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop with progress bar
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training', leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)
        
        train_accuracy = 100 * correct_train / total_train
        train_loss = train_loss / len(train_loader.dataset)
        
        # Validation loop
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        model.eval()
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                
                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)
        
        val_accuracy = 100 * correct_val / total_val
        val_loss = val_loss / len(val_loader.dataset)
        
        # Print metrics for the epoch
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
        
        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()
    
    # Load the best model before returning
    model.load_state_dict(best_model)
    print(f"Best Validation Accuracy: {best_val_accuracy:.2f}%")
    return model


In [8]:
model=train_model(model,train_loader,val_loader,criterion,optimizer)

Epoch 1/50 - Training:   0%|          | 0/22 [00:00<?, ?it/s]/tmp/ipykernel_1891/3210615150.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


Epoch 1/50
  Train Loss: 2.7466, Train Accuracy: 12.39%
  Val Loss: 4.2214, Val Accuracy: 6.46%


Epoch 2/50
  Train Loss: 2.0306, Train Accuracy: 23.21%
  Val Loss: 3.7066, Val Accuracy: 7.38%


Epoch 3/50
  Train Loss: 1.6876, Train Accuracy: 32.26%
  Val Loss: 2.2691, Val Accuracy: 17.88%


Epoch 4/50
  Train Loss: 1.5364, Train Accuracy: 37.26%
  Val Loss: 2.2174, Val Accuracy: 19.08%


...

Epoch 46/50
  Train Loss: 0.5164, Train Accuracy: 78.62%
  Val Loss: 1.0569, Val Accuracy: 58.83%


Epoch 47/50
  Train Loss: 0.4610, Train Accuracy: 81.64%
  Val Loss: 0.5167, Val Accuracy: 87.79%


Epoch 48/50
  Train Loss: 0.3999, Train Accuracy: 84.59%
  Val Loss: 0.5314, Val Accuracy: 85.83%


Epoch 49/50
  Train Loss: 0.3215, Train Accuracy: 88.40%
  Val Loss: 0.4558, Val Accuracy: 92.75%


Epoch 50/50
  Train Loss: 0.3098, Train Accuracy: 88.63%
  Val Loss: 0.4754, Val Accuracy: 90.58%
Best Validation Accuracy: 92.75%


In [9]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate_model(model, test_loader)


/tmp/ipykernel_1891/3210615150.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


Test Accuracy: 90.30%


In [10]:
# Save the model after training
torch.save(model.state_dict(), "part2.pth")
print("Model saved to part2.pth")

Model saved to part2.pth
